
# Phase 4.1 — End-to-End System Evaluation & Performance Profiling

## Purpose

Evaluate the full RAG-IDS pipeline on a **reserved sequential test slice** of the `ocean_v51` corpus. No shuffling — temporal order within each archetype partition is preserved to maintain Phase 3.1 sliding-window integrity.

## Evaluation pipeline

```
ocean_v51 (sequential slice, ≥50,000 rows)
  → vectorize_v51            [Phase 2.3 — 114-dim behavioral space]
  → get_rag_context          [Phase 2.3 — ChromaDB HNSW cosine retrieval]
  → AdaptiveTimeWindow       [Phase 3.1 — deque buffer, linear ±steps]
  → accumulate_evidence      [Phase 3.2 — Dempster-Shafer sequential fusion]
  → quantum_fuse             [Phase 3.3 — H_γ interference → P_fused]
  → generate_security_report [Phase 3.4 — deterministic alert]
  ──────────────────────────────────────────────────────────────────────
  → Classification Metrics   [Phase 4.1 — Accuracy, Precision, Recall, F1]
  → Confusion Matrix
  → Latency & Throughput
  → Quantum Advantage (K > 0.50 edge cases)
```

## Binary mapping

| Verdict | Prediction |
|---|---|
| CRITICAL, WARNING, ELEVATED | 1 (Attack) |
| MONITOR, CLEAR | 0 (Normal) |

| Archetype | Ground Truth |
|---|---|
| NORMAL | 0 |
| BOTNET_C2, BRUTE_FORCE, DOS_DDOS, EXPLOIT, SCAN, THEFT_EXFIL | 1 |


In [1]:

# ── Cell 1: Imports ───────────────────────────────────────────────────────────

import sys, os, gc, time, math, pickle, warnings, json, functools
from pathlib import Path
from dataclasses import dataclass, field
from collections import deque
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

import pyarrow.parquet as pq
import chromadb
from chromadb import PersistentClient

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
)

SEP  = '─' * 78
SEP2 = '═' * 78

print(f'Python    : {sys.version.split()[0]}')
print(f'NumPy     : {np.__version__}')
print(f'Pandas    : {pd.__version__}')
print(f'ChromaDB  : {chromadb.__version__}')
print(f'scikit-learn imported ✅')
print(f'functools  imported ✅')
print('Phase 4.2 — System Hardening imports ready.')


Python    : 3.13.9
NumPy     : 2.1.3
Pandas    : 2.2.3
ChromaDB  : 1.4.1
scikit-learn imported ✅
functools  imported ✅
Phase 4.2 — System Hardening imports ready.


In [2]:

# ── Cell 2: Paths, Preprocessors & ChromaDB ───────────────────────────────────

NOTEBOOK_DIR    = Path.cwd()
MAIN_DIR        = NOTEBOOK_DIR.parent
DATA_DIR        = MAIN_DIR / 'data'
ARTIFACTS_DIR   = MAIN_DIR / 'artifacts'
OCEAN_DIR       = DATA_DIR / 'unified' / 'ocean_v51'
CHROMA_DIR      = MAIN_DIR / 'chromadb_store_v51'
COLLECTION_NAME = 'ids_knowledge_base_v51'

PREPROCESSORS_PATH = ARTIFACTS_DIR / 'preprocessors_v51.pkl'

# ── Load Preprocessors ─────────────────────────────────────────────────────────
assert PREPROCESSORS_PATH.exists(), f'MISSING: {PREPROCESSORS_PATH}'
print(f'Loading {PREPROCESSORS_PATH.name} …')
with open(PREPROCESSORS_PATH, 'rb') as _f:
    PP = pickle.load(_f)

block1_scalers   = PP['block1_scalers']
block6_scalers   = PP['block6_scalers']
qt_byte_pkt      = PP['qt_byte_pkt']
pt_sport         = PP['pt_sport_rarity']
pt_dport         = PP['pt_dport_rarity']
sport_rarity     = PP['sport_rarity_map']
dport_rarity     = PP['dport_rarity_map']
TOTAL_ROWS_OCEAN = PP['total_rows_ocean']

print(f'  Preprocessors loaded : {len(PP)} keys')
print(f'  Total ocean rows     : {TOTAL_ROWS_OCEAN:,}')

# ── Connect to ChromaDB ────────────────────────────────────────────────────────
assert CHROMA_DIR.exists(), f'MISSING ChromaDB store: {CHROMA_DIR}'
print(f'\nConnecting to ChromaDB at {CHROMA_DIR.name} …')
_client    = PersistentClient(path=str(CHROMA_DIR))
collection = _client.get_collection(COLLECTION_NAME)

print(f'  Collection : {collection.name}')
print(f'  Medoids    : {collection.count():,}')
print('\nDependencies ready ✅')


Loading preprocessors_v51.pkl …
  Preprocessors loaded : 15 keys
  Total ocean rows     : 351,317,489

Connecting to ChromaDB at chromadb_store_v51 …
  Collection : ids_knowledge_base_v51
  Medoids    : 102,505

Dependencies ready ✅


In [3]:

# ── Cell 3: vectorize_v51 + get_rag_context (exact Phase 2.3 replicas) ─────────
#
# These replicas are bit-for-bit identical to Phase 2.3 — mandatory so that
# query vectors live in the same geometric space as the stored medoids.
#
# Phase 4.3 addition: Absolute Similarity Floor in get_rag_context_batch().
#   If top similarity < RAG_SIM_FLOOR (75%), override to UNKNOWN / 0.0.
#   Prevents low-confidence RAG matches from contaminating the DS accumulator.
# ─────────────────────────────────────────────────────────────────────────────

TOTAL_DIMS = 114
RAG_SIM_FLOOR = 75.0    # Phase 4.3 — similarity floor for RAG trust

PROTO_TOKENS     = ['tcp', 'udp', 'icmp', 'arp', 'ipv6', 'other']
SERVICE_TOKENS   = ['dns', 'http', 'ssl', 'ftp', 'ssh', 'smtp',
                    'dhcp', 'quic', 'ntp', 'rdp', 'pop3', 'other']
STATE_TOKENS     = ['PENDING', 'ESTABLISHED', 'REJECTED', 'RESET', 'OTHER']
PORT_FUNC_TOKENS = ['SCADA_CONTROL', 'IOT_MANAGEMENT', 'WEB_SERVICES',
                    'NETWORK_CORE',  'REMOTE_ACCESS',  'FUNC_EPHEMERAL', 'FUNC_UNKNOWN']
HTTP_METHOD_TOKENS = ['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'OTHER']
SSL_CIPHER_TOKENS  = [
    'TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256', 'TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384',
    'TLS_RSA_WITH_AES_128_GCM_SHA256',       'TLS_RSA_WITH_AES_256_GCM_SHA384',
    'TLS_RSA_WITH_AES_128_CBC_SHA',          'TLS_RSA_WITH_AES_256_CBC_SHA',
    'TLS_RSA_WITH_RC4_128_SHA',              'TLS_RSA_WITH_RC4_128_MD5',
    'TLS_RSA_WITH_3DES_EDE_CBC_SHA',         'TLS_DHE_RSA_WITH_AES_128_CBC_SHA',
    'TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256', 'other',
]
SCADA_PORTS    = frozenset({502, 102, 44818})
IOT_MGMT_PORTS = frozenset({1883, 5683, 8883})
WEB_PORTS      = frozenset({80, 443, 8080})
NET_CORE_PORTS = frozenset({53, 67, 68, 123})
REMOTE_PORTS   = frozenset({22, 23, 3389})
_PROTO_IDX   = {t: i for i, t in enumerate(PROTO_TOKENS)}
_SVC_IDX     = {t: i for i, t in enumerate(SERVICE_TOKENS)}
_STATE_IDX   = {t: i for i, t in enumerate(STATE_TOKENS)}
_METHOD_IDX  = {t: i for i, t in enumerate(HTTP_METHOD_TOKENS)}
_CIPHER_IDX  = {t: i for i, t in enumerate(SSL_CIPHER_TOKENS)}
_ABSENT_SVCS = frozenset({'<absent>', '-', 'unknown', '', 'none', '(empty)', 'nan'})
_DNS_QTYPE_MAP  = {1:0, 2:1, 5:2, 6:3, 12:4, 15:5, 16:6, 28:7, 33:8, 255:9}
_WEAK_SSL_VER   = frozenset({'sslv2','sslv3','tlsv1','tlsv10','tlsv1.0','tls1.0'})
_STRONG_SSL_VER = frozenset({'tlsv12','tlsv13','tlsv1.2','tlsv1.3','tls1.2','tls1.3'})

def _classify_port_vec(port_series):
    p = pd.to_numeric(port_series, errors='coerce').fillna(-1).astype(int)
    result = np.full(len(p), 6, dtype=np.int8)
    for _, func_idx, pset in [(0,0,SCADA_PORTS),(1,1,IOT_MGMT_PORTS),(2,2,WEB_PORTS),
                               (3,3,NET_CORE_PORTS),(4,4,REMOTE_PORTS)]:
        result[p.isin(pset).values] = func_idx
    result[(p.values > 49152) & (result == 6)] = 5
    return result

def vectorize_v51(df):
    """Map a DataFrame of v5.1-aligned ocean rows → (N, 114) float32."""
    n = len(df); X = np.zeros((n, TOTAL_DIMS), dtype=np.float32); idx = df.index
    def _col(name, fill=0.0):
        return df[name].fillna(fill) if name in df.columns else pd.Series(fill, index=idx)
    def _str_col(name, fill=''):
        return (df[name].fillna(fill).astype(str).str.lower().str.strip()
                if name in df.columns else pd.Series(fill, index=idx))
    for col, mode, out_i in [('univ_duration','rs',0),('univ_bytes_in','qt',1),
                              ('univ_bytes_out','qt',2),('univ_pkts_in','qt',3),
                              ('univ_pkts_out','qt',4)]:
        vals = np.clip(_col(col,0.).values.astype(np.float64), 0., None)
        if mode=='qt' and col in qt_byte_pkt:
            X[:,out_i] = qt_byte_pkt[col].transform(vals.reshape(-1,1)).ravel().astype(np.float32)
        elif mode=='rs' and col in block1_scalers:
            X[:,out_i] = block1_scalers[col].transform(np.log1p(vals).reshape(-1,1)).ravel().astype(np.float32)
    proto = _str_col('raw_proto','other')
    X[np.arange(n), 5 + proto.map(lambda p: _PROTO_IDX.get(p, _PROTO_IDX['other'])).values] = 1.
    has_svc = _col('has_svc',0).values.astype(np.float32)
    svc = _str_col('raw_service','other')
    other_svc = _SVC_IDX['other']
    svc_idx = svc.map(lambda s: _SVC_IDX.get(s,other_svc) if s not in _ABSENT_SVCS else other_svc).values
    svc_ohe = np.zeros((n,12), dtype=np.float32); svc_ohe[np.arange(n), svc_idx] = 1.
    X[:,11:23] = svc_ohe * has_svc[:,np.newaxis]
    state = (df['raw_state_v51'].fillna('OTHER').astype(str).str.upper()
             if 'raw_state_v51' in df.columns else pd.Series('OTHER',index=idx))
    X[np.arange(n), 23 + state.map(lambda s: _STATE_IDX.get(s,_STATE_IDX['OTHER'])).values] = 1.
    X[np.arange(n), 28 + _classify_port_vec(_col('raw_sport',-1))] = 1.
    X[np.arange(n), 35 + _classify_port_vec(_col('raw_dport',-1))] = 1.
    DEFAULT_R = 1. / max(TOTAL_ROWS_OCEAN, 1)
    sport_str = _col('raw_sport',-1).values.astype(int).astype(str)
    dport_str = _col('raw_dport',-1).values.astype(int).astype(str)
    sr = np.array([sport_rarity.get(p, DEFAULT_R) for p in sport_str], dtype=np.float64)
    dr = np.array([dport_rarity.get(p, DEFAULT_R) for p in dport_str], dtype=np.float64)
    if pt_sport: X[:,42] = pt_sport.transform(sr.reshape(-1,1)).ravel().astype(np.float32)
    if pt_dport: X[:,43] = pt_dport.transform(dr.reshape(-1,1)).ravel().astype(np.float32)
    has_dns = _col('has_dns',0).values.astype(np.float32)
    qtype = _col('dns_qtype',-1).values.astype(int)
    qclass = _col('dns_qclass',-1).values.astype(int)
    rcode  = _col('dns_rcode',-1).values.astype(int)
    qt_arr = np.zeros((n,10), dtype=np.float32)
    for code, qi in _DNS_QTYPE_MAP.items(): qt_arr[qtype==code, qi] = 1.
    qt_arr[(qtype>0) & ~np.isin(qtype, list(_DNS_QTYPE_MAP.keys())), 9] = 1.
    X[:,44:54] = qt_arr * has_dns[:,np.newaxis]
    qc_arr = np.zeros((n,3), dtype=np.float32)
    qc_arr[qclass==1,0]=1.; qc_arr[qclass==3,1]=1.
    qc_arr[(qclass>=0)&(qclass!=1)&(qclass!=3),2]=1.
    X[:,54:57] = qc_arr * has_dns[:,np.newaxis]
    X[:,57] = ((rcode==0)&(has_dns>0)).astype(np.float32)
    X[:,58] = ((rcode>0)&(has_dns>0)).astype(np.float32)
    has_http = _col('has_http',0).values.astype(np.float32)
    http_m = (df['raw_http_method'].fillna('-').astype(str).str.strip().str.upper()
              if 'raw_http_method' in df.columns else pd.Series('-',index=idx))
    m_idx = http_m.map(lambda m: _METHOD_IDX.get(m,_METHOD_IDX['OTHER'])).values
    m_arr = np.zeros((n,8), dtype=np.float32)
    valid_m = (http_m!='-')&(http_m!='')&(http_m!='NAN')
    m_arr[valid_m.values, m_idx[valid_m.values]] = 1.
    X[:,59:67] = m_arr * has_http[:,np.newaxis]
    http_s = _col('http_status_code',-1).values.astype(int)
    s_arr = np.zeros((n,6), dtype=np.float32)
    s_arr[(http_s>=100)&(http_s<200),0]=1.; s_arr[(http_s>=200)&(http_s<300),1]=1.
    s_arr[(http_s>=300)&(http_s<400),2]=1.; s_arr[(http_s>=400)&(http_s<500),3]=1.
    s_arr[(http_s>=500)&(http_s<600),4]=1.; s_arr[http_s<0,5]=1.
    X[:,67:73] = s_arr * has_http[:,np.newaxis]
    req_b  = np.clip(_col('http_req_body_len',0).values.astype(np.float64),0,1e7)
    resp_b = np.clip(_col('http_resp_body_len',0).values.astype(np.float64),0,1e7)
    X[:,73] = (np.log1p(req_b)/np.log1p(1e7)).astype(np.float32)*has_http
    X[:,74] = (np.log1p(resp_b)/np.log1p(1e7)).astype(np.float32)*has_http
    X[:,75] = valid_m.values.astype(np.float32)*has_http
    X[:,76] = (http_s>=100).astype(np.float32)*has_http
    X[:,77] = (req_b>0).astype(np.float32)*has_http
    X[:,78] = (resp_b>0).astype(np.float32)*has_http
    has_ssl = _col('has_ssl',0).values.astype(np.float32)
    ssl_c = (df['raw_ssl_cipher'].fillna('').astype(str).str.strip()
             if 'raw_ssl_cipher' in df.columns else pd.Series('',index=idx))
    c_arr = np.zeros((n,12), dtype=np.float32)
    c_arr[np.arange(n), ssl_c.map(lambda c: _CIPHER_IDX.get(c,_CIPHER_IDX['other'])).values] = 1.
    X[:,80:92] = c_arr * has_ssl[:,np.newaxis]
    ssl_v = (df['raw_ssl_version'].fillna('').astype(str).str.strip().str.lower()
              .str.replace(' ','').str.replace('.','') if 'raw_ssl_version' in df.columns
              else pd.Series('',index=idx))
    X[:,92] = ssl_v.isin(_WEAK_SSL_VER).values.astype(np.float32)*has_ssl
    X[:,93] = ssl_v.isin(_STRONG_SSL_VER).values.astype(np.float32)*has_ssl
    X[:,94] = _col('ssl_established',0).values.astype(np.float32)*has_ssl
    has_unsw = _col('has_unsw',0).values.astype(np.float32)
    BLOCK6 = ['mom_mean','mom_stddev','mom_sum','mom_min','mom_max','mom_rate',
               'mom_srate','mom_drate','mom_TnBPSrcIP','mom_TnBPDstIP',
               'mom_TnP_PSrcIP','mom_TnP_PDstIP','mom_TnP_PerProto','mom_TnP_Per_Dport']
    for i, col in enumerate(BLOCK6):
        if col in block6_scalers:
            info = block6_scalers[col]; rs = info['scaler']; shift = info['shift']
            vals = _col(col,-1.).values.astype(np.float64); valid = vals != -1.
            out  = np.zeros(n, dtype=np.float32)
            if valid.any():
                out[valid] = rs.transform(np.log1p(vals[valid]+shift).reshape(-1,1)).ravel().astype(np.float32)
            X[:,95+i] = out * has_unsw
    X[:,109]=has_svc; X[:,110]=has_dns; X[:,111]=has_http; X[:,112]=has_ssl; X[:,113]=has_unsw
    np.nan_to_num(X, nan=0., posinf=0., neginf=0., copy=False)
    return X

def _l2_normalise(X: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms = np.where(norms == 0., 1., norms)
    return (X / norms).astype(np.float32)

def get_rag_context_batch(X_norm: np.ndarray, n_results: int = 3) -> list:
    """
    Query ChromaDB for a batch of pre-normalised vectors.

    Phase 4.3 — Absolute Similarity Floor:
      If the top neighbour's similarity_pct < RAG_SIM_FLOOR (75%),
      the packet is too far from any known medoid.  Override to UNKNOWN
      so the downstream BPA maps it to pure uncertainty (m_U = 1.0).
    """
    results = collection.query(
        query_embeddings=X_norm.tolist(),
        n_results=n_results,
        include=['metadatas', 'distances'],
    )
    outputs = []
    for i in range(len(X_norm)):
        neighbours = []
        for rank, (meta, dist) in enumerate(
            zip(results['metadatas'][i], results['distances'][i]), start=1
        ):
            dist_c  = max(0., float(dist))
            sim_pct = (1. - dist_c / 2.) * 100.
            neighbours.append({
                'rank'          : rank,
                'archetype'     : meta.get('ubt_archetype',       'UNKNOWN'),
                'attack_variant': meta.get('univ_specific_attack', 'UNKNOWN'),
                'dataset_source': meta.get('dataset_source',       'UNKNOWN'),
                'distance'      : round(dist_c, 6),
                'similarity_pct': round(sim_pct, 2),
            })
        top = neighbours[0] if neighbours else {}
        top_sim = top.get('similarity_pct', 0.)

        # ── Phase 4.3: Absolute Similarity Floor ──────────────────────────────
        if top_sim < RAG_SIM_FLOOR:
            outputs.append({
                'top_archetype' : 'UNKNOWN',
                'top_attack'    : 'UNKNOWN',
                'top_similarity': 0.0,
                'top_distance'  : top.get('distance', 2.0),
            })
        else:
            outputs.append({
                'top_archetype' : top.get('archetype',      'UNKNOWN'),
                'top_attack'    : top.get('attack_variant', 'UNKNOWN'),
                'top_similarity': top_sim,
                'top_distance'  : top.get('distance',       2.0),
            })
    return outputs

print('vectorize_v51 and get_rag_context_batch defined ✅')
print(f'  Output shape: (N, {TOTAL_DIMS}) float32  — v5.1 schema lock')
print(f'  RAG similarity floor: {RAG_SIM_FLOOR}% (Phase 4.3)')


vectorize_v51 and get_rag_context_batch defined ✅
  Output shape: (N, 114) float32  — v5.1 schema lock
  RAG similarity floor: 75.0% (Phase 4.3)


In [5]:

# ── Cell 4: Phase 4.2/4.3/4.4 — Hardened Phase 3 Engine Replicas ─────────────
#
# Target 1 — AdaptiveTimeWindow: "Clean Slate" protocol
#   _benign_streak counter; buffer flushed after 3 consecutive high-confidence
#   benign flows (top_archetype ∈ NORMAL_ARCHETYPES AND top_similarity ≥ 85%).
#   Phase 4.4: ingest() uses amortised O(1) popleft instead of O(N) filter.
#
# Target 2 — Vectorized BPA + Dempster-Shafer + UNKNOWN mask:
#   compute_bpa_batch() → (N, 3) NumPy array; UNKNOWN → pure uncertainty.
#   accumulate_evidence() → scalar Dempster fold.
#
# Target 3 (Phase 4.3) — Conflict-Weighted Quantum-Classical Blend:
#   P_final = K × P_quantum + (1 − K) × P_classical
#
# Target 4 (Phase 4.4) — π/2 phase encoding:
#   Bound quantum phase rotation from π to π/2 to prevent complete
#   amplitude inversion during high-conflict edge cases.
# ─────────────────────────────────────────────────────────────────────────────

VECTOR_DIMS      = 114
THRESH_CRITICAL  = 0.90
THRESH_WARNING   = 0.70
THRESH_CLEAR     = 0.30
THRESH_MONITOR_U = 0.40
CONFLICT_EPS     = 1e-9
MIN_UNCERTAINTY  = 1e-6
NORMAL_ARCHETYPES = frozenset({'NORMAL', 'benign', 'normal'})

# ── FlowRecord ─────────────────────────────────────────────────────────────────
@dataclass
class FlowRecord:
    timestamp     : float
    vector        : np.ndarray
    top_similarity: float
    top_archetype : str
    top_attack    : str
    packet_label  : str
    rag_context   : str = ''

    def __post_init__(self):
        arr = np.asarray(self.vector, dtype=np.float32)
        if arr.shape != (VECTOR_DIMS,):
            raise ValueError(f'vector must be ({VECTOR_DIMS},), got {arr.shape}')
        object.__setattr__(self, 'vector', arr)

# ── Target 1: AdaptiveTimeWindow — Clean Slate Protocol ───────────────────────
@dataclass
class AdaptiveTimeWindow:
    """
    Deque-backed sliding window with Clean Slate protocol.

    Clean Slate: After BENIGN_STREAK_THRESHOLD consecutive flows whose
    top_archetype ∈ NORMAL_ARCHETYPES AND top_similarity ≥ BENIGN_SIM_THRESHOLD,
    the buffer and window size are reset to their initial state.
    """
    min_size_s             : float = 5.0
    max_size_s             : float = 30.0
    step_up_s              : float = 2.0
    step_down_s            : float = 1.0
    max_buffer             : int   = 1000
    BENIGN_STREAK_THRESHOLD: int   = 3
    BENIGN_SIM_THRESHOLD   : float = 85.0

    _current_size_s : float = field(init=False)
    _buffer         : deque = field(init=False)
    _benign_streak  : int   = field(init=False)

    def __post_init__(self):
        self._current_size_s = self.min_size_s
        self._buffer         = deque(maxlen=self.max_buffer)
        self._benign_streak  = 0

    @property
    def window_size_s(self) -> float:
        return self._current_size_s

    def ingest(self, flow: FlowRecord) -> List[FlowRecord]:
        # ── Streak evaluation ──────────────────────────────────────────────────
        if (flow.top_archetype in NORMAL_ARCHETYPES
                and flow.top_similarity >= self.BENIGN_SIM_THRESHOLD):
            self._benign_streak += 1
        else:
            self._benign_streak = 0

        # ── Clean Slate trigger ────────────────────────────────────────────────
        if self._benign_streak >= self.BENIGN_STREAK_THRESHOLD:
            self._buffer.clear()
            self._current_size_s = self.min_size_s
            self._benign_streak  = 0
            self._buffer.append(flow)
            return list(self._buffer)

        self._buffer.append(flow)
        cutoff = flow.timestamp - self._current_size_s
        # Phase 4.4: amortised O(1) left-pop instead of O(N) list comprehension
        while self._buffer and self._buffer[0].timestamp < cutoff:
            self._buffer.popleft()
        return list(self._buffer)

    def adapt(self, verdict: str) -> None:
        if verdict in ('CRITICAL', 'WARNING'):
            self._current_size_s = min(
                self._current_size_s + self.step_up_s, self.max_size_s)
        elif verdict == 'CLEAR':
            self._current_size_s = max(
                self._current_size_s - self.step_down_s, self.min_size_s)

# ── Target 2a: Vectorized BPA computation → (N, 3) NumPy array ────────────────
def compute_bpa_batch(buffer: List[FlowRecord]) -> np.ndarray:
    """
    Vectorised BPA with UNKNOWN mask (Phase 4.3).
    UNKNOWN → pure uncertainty: m_A=0, m_N=0, m_U=1.
    Returns float64 (N, 3) array — columns: [m_A, m_N, m_U], rows sum to 1.
    """
    sims  = np.fromiter((f.top_similarity for f in buffer),
                        dtype=np.float64, count=len(buffer))
    archs = [f.top_archetype for f in buffer]

    sim_n      = np.clip(sims, 0., 100.) / 100.
    is_normal  = np.array([a in NORMAL_ARCHETYPES for a in archs], dtype=bool)
    is_unknown = np.array([a == 'UNKNOWN' for a in archs], dtype=bool)

    m_A = np.where(is_normal, sim_n * 0.05, sim_n * 0.95)
    m_N = np.where(is_normal, sim_n * 0.90, sim_n * 0.02)
    m_U = np.maximum(MIN_UNCERTAINTY, 1.0 - m_A - m_N)

    # Phase 4.3: UNKNOWN → pure uncertainty
    m_A[is_unknown] = 0.0
    m_N[is_unknown] = 0.0
    m_U[is_unknown] = 1.0

    totals = m_A + m_N + m_U
    m_A /= totals;  m_N /= totals;  m_U /= totals

    return np.column_stack([m_A, m_N, m_U])

# ── Target 2b: accumulate_evidence — scalar loop over pre-computed BPA matrix ─
def accumulate_evidence(buffer: List[FlowRecord]) -> Dict[str, float]:
    """
    Phase 4.2 — vectorised BPA + scalar Dempster fold.
    """
    bpas = compute_bpa_batch(buffer)

    m_A, m_N, m_U = float(bpas[0, 0]), float(bpas[0, 1]), float(bpas[0, 2])
    cum_K = 0.0

    for i in range(1, len(bpas)):
        b2_A = float(bpas[i, 0])
        b2_N = float(bpas[i, 1])
        b2_U = float(bpas[i, 2])

        K     = m_A * b2_N + b2_A * m_N
        denom = 1.0 - K
        if K > cum_K:
            cum_K = K

        if denom < CONFLICT_EPS:
            m_A, m_N, m_U = 0.5, 0.5, MIN_UNCERTAINTY
            continue

        new_A = (m_A * b2_A  +  m_A * b2_U  +  b2_A * m_U) / denom
        new_N = (m_N * b2_N  +  m_N * b2_U  +  b2_N * m_U) / denom
        new_U = (m_U * b2_U) / denom
        m_A   = new_A
        m_N   = new_N
        m_U   = new_U if new_U > MIN_UNCERTAINTY else MIN_UNCERTAINTY

    return {
        'belief_attack' : round(m_A,   8),
        'belief_normal' : round(m_N,   8),
        'uncertainty'   : round(m_U,   8),
        'conflict_level': round(cum_K, 8),
    }

# ── Quantum Fusion — Phase 4.3/4.4: Conflict-Weighted Blend + π/2 Encoding ────
_VERDICT_ICONS = {'CRITICAL':'🔴','WARNING':'🟠','ELEVATED':'🟡','MONITOR':'⚪','CLEAR':'🟢'}

def generate_ids_verdict(P: float, U: float) -> Dict:
    if   P >= THRESH_CRITICAL: v='CRITICAL'; desc=f'P={P:.4f}≥{THRESH_CRITICAL}'
    elif P >= THRESH_WARNING:  v='WARNING';  desc=f'P={P:.4f}≥{THRESH_WARNING}'
    elif U > THRESH_MONITOR_U: v='MONITOR';  desc=f'U={U:.4f}>{THRESH_MONITOR_U}'
    elif P < THRESH_CLEAR:     v='CLEAR';    desc=f'P={P:.4f}<{THRESH_CLEAR}'
    else:                       v='ELEVATED'; desc=f'P={P:.4f} elevated'
    return {'verdict':v, 'icon':_VERDICT_ICONS.get(v,'⚫'),
            'confidence':round(1.-U,4), 'description':desc}

def quantum_fuse(evidence: Dict) -> Dict:
    """
    Quantum Knowledge Fusion with Phase 4.3 conflict-weighted blend
    and Phase 4.4 π/2 phase encoding.

    Phase 4.3: Blend quantum output with classical DS evidence weighted by K:
        P_final = K × P_quantum + (1 − K) × m_A
    Phase 4.4: Bound phase rotation from π to π/2 to prevent complete
    amplitude inversion in high-conflict edge cases.
    """
    m_A = evidence['belief_attack'];  m_N = evidence['belief_normal']
    m_U = evidence['uncertainty'];    K   = evidence['conflict_level']

    # ── Quantum state preparation ─────────────────────────────────────────────
    # Phase 4.4: bound rotation to π/2 to prevent complete amplitude inversion
    alpha = math.sqrt(max(m_A, 0.)) * np.exp(1j * (math.pi / 2.0) * m_U)
    beta  = math.sqrt(max(m_N, 0.)) * np.exp(1j * (math.pi / 2.0) * K)
    psi   = np.array([alpha, beta], dtype=np.complex128)
    norm  = np.linalg.norm(psi)
    if norm > 1e-15:
        psi /= norm
    prob_atk_raw = float(np.abs(psi[0])**2)

    # ── Hadamard interference gate ────────────────────────────────────────────
    gamma   = 1.0 - K
    inv_rt2 = 1. / math.sqrt(2.)
    H       = inv_rt2 * np.array([[1., gamma], [gamma, -1.]], dtype=np.complex128)
    psi_p   = H @ psi
    np_     = np.linalg.norm(psi_p)
    if np_ > 1e-15:
        psi_p /= np_
    p_atk_quantum = float(np.abs(psi_p[0])**2)
    p_nrm_quantum = float(np.abs(psi_p[1])**2)

    # ── Phase 4.3: Conflict-weighted quantum-classical blend ──────────────────
    #    Low K  → trust classical DS directly (no false Hadamard inflation).
    #    High K → trust quantum disambiguation.
    p_atk = K * p_atk_quantum + (1.0 - K) * m_A
    p_nrm = 1.0 - p_atk

    itype = ('CONSTRUCTIVE' if p_atk > prob_atk_raw + 0.01
             else ('DESTRUCTIVE' if p_atk < prob_atk_raw - 0.01 else 'MIXED'))
    vd = generate_ids_verdict(p_atk, m_U)
    return {**evidence,
            'fused_probability': round(p_atk, 6),
            'prob_normal_post' : round(p_nrm, 6),
            'coherence_factor' : round(gamma, 6),
            'interference_type': itype,
            'quantum_raw'      : round(p_atk_quantum, 6),
            'verdict'          : vd['verdict'],
            'icon'             : vd['icon'],
            'confidence'       : vd['confidence']}

# ── Smoke-tests ────────────────────────────────────────────────────────────────
def _make_flow(arch, sim):
    f = FlowRecord(0., np.zeros(114), sim, arch, 'none', arch)
    return f

# Clean Slate smoke-test
_atw_test = AdaptiveTimeWindow()
for _ in range(3):
    _buf_out = _atw_test.ingest(_make_flow('NORMAL', 90.))
assert len(_buf_out) == 1, 'Expected single-element buffer after Clean Slate flush'
print(f'Clean Slate smoke-test: buffer after 3×high-sim benign = {len(_buf_out)} flow ✅')

# Vectorised BPA smoke-test
_test_buf = [_make_flow('BOTNET_C2', 80.), _make_flow('NORMAL', 90.)]
_bpas     = compute_bpa_batch(_test_buf)
assert _bpas.shape == (2, 3) and np.allclose(_bpas.sum(axis=1), 1.)
print(f'Vectorised BPA smoke-test: shape={_bpas.shape},  row-sums≈1 ✅')

# Phase 4.3 — UNKNOWN mask smoke-test
_unk_buf = [_make_flow('UNKNOWN', 0.0), _make_flow('BOTNET_C2', 95.)]
_unk_bpas = compute_bpa_batch(_unk_buf)
assert _unk_bpas[0, 0] == 0.0 and _unk_bpas[0, 1] == 0.0 and _unk_bpas[0, 2] == 1.0, \
    f'UNKNOWN flow must be pure uncertainty, got {_unk_bpas[0]}'
print(f'UNKNOWN mask smoke-test: m_A={_unk_bpas[0,0]:.1f}, m_N={_unk_bpas[0,1]:.1f}, m_U={_unk_bpas[0,2]:.1f} ✅')

# Phase 4.3 — Quantum blend smoke-test (NORMAL should map to CLEAR)
_norm_buf = [_make_flow('NORMAL', 100.)]
_norm_ev  = accumulate_evidence(_norm_buf)
_norm_qf  = quantum_fuse(_norm_ev)
print(f'Quantum blend smoke-test (NORMAL@100%): P_fused={_norm_qf["fused_probability"]:.4f}, '
      f'quantum_raw={_norm_qf["quantum_raw"]:.4f}, verdict={_norm_qf["verdict"]} ✅')
assert _norm_qf['verdict'] in ('CLEAR', 'MONITOR'), \
    f'NORMAL@100% should be CLEAR/MONITOR, got {_norm_qf["verdict"]}'

# Phase 4.3 — Quantum blend smoke-test (ATTACK should stay CRITICAL)
_atk_buf = [_make_flow('BOTNET_C2', 95.)]
_atk_ev  = accumulate_evidence(_atk_buf)
_atk_qf  = quantum_fuse(_atk_ev)
print(f'Quantum blend smoke-test (ATTACK@95%): P_fused={_atk_qf["fused_probability"]:.4f}, '
      f'quantum_raw={_atk_qf["quantum_raw"]:.4f}, verdict={_atk_qf["verdict"]} ✅')
assert _atk_qf['verdict'] == 'CRITICAL', \
    f'ATTACK@95% should be CRITICAL, got {_atk_qf["verdict"]}'

# accumulate_evidence returns valid dict keys
_ev = accumulate_evidence(_test_buf)
assert set(_ev) == {'belief_attack','belief_normal','uncertainty','conflict_level'}
print(f'accumulate_evidence smoke-test: {_ev} ✅')

print()
print('Phase 4.2/4.3/4.4 hardened engine loaded ✅')
print('  ✅ AdaptiveTimeWindow — Clean Slate (streak ≥ 3 @ sim ≥ 85%)')
print('  ✅ Phase 4.4: ingest() amortised O(1) popleft')
print('  ✅ compute_bpa_batch  — vectorised NumPy (N×3) + UNKNOWN mask')
print('  ✅ accumulate_evidence — vectorised BPA + scalar Dempster fold')
print('  ✅ Phase 4.3: UNKNOWN archetype → pure uncertainty (FPR fix)')
print('  ✅ Phase 4.3: Conflict-weighted quantum-classical blend (FPR fix)')
print('  ✅ Phase 4.4: π/2 phase encoding (softened destructive interference)')


print('  ✅ Phase 4.3: UNKNOWN archetype → pure uncertainty (FPR fix)')
print('  ✅ Phase 4.3: Conflict-weighted quantum-classical blend (FPR fix)')

Clean Slate smoke-test: buffer after 3×high-sim benign = 1 flow ✅
Vectorised BPA smoke-test: shape=(2, 3),  row-sums≈1 ✅
UNKNOWN mask smoke-test: m_A=0.0, m_N=0.0, m_U=1.0 ✅
Quantum blend smoke-test (NORMAL@100%): P_fused=0.0500, quantum_raw=0.7226, verdict=CLEAR ✅
Quantum blend smoke-test (ATTACK@95%): P_fused=0.9025, quantum_raw=0.6410, verdict=CRITICAL ✅
accumulate_evidence smoke-test: {'belief_attack': 0.40262719, 'belief_normal': 0.51271893, 'uncertainty': 0.08465388, 'conflict_level': 0.61632} ✅

Phase 4.2/4.3/4.4 hardened engine loaded ✅
  ✅ AdaptiveTimeWindow — Clean Slate (streak ≥ 3 @ sim ≥ 85%)
  ✅ Phase 4.4: ingest() amortised O(1) popleft
  ✅ compute_bpa_batch  — vectorised NumPy (N×3) + UNKNOWN mask
  ✅ accumulate_evidence — vectorised BPA + scalar Dempster fold
  ✅ Phase 4.3: UNKNOWN archetype → pure uncertainty (FPR fix)
  ✅ Phase 4.3: Conflict-weighted quantum-classical blend (FPR fix)
  ✅ Phase 4.4: π/2 phase encoding (softened destructive interference)
  ✅ Phase 4.3:

In [6]:

# ── Cell 5: Load Sequential Test Slice ────────────────────────────────────────
#
# Phase 4.4: 8,400 rows × 7 archetypes → 50,497 total (THEFT_EXFIL capped at 97).
#   • For each archetype, list sorted parquet files and TAIL from the last file(s).
#   • No shuffle — preserves natural temporal ordering within each archetype.
#   • vectorize_v51 reads whichever columns it needs; all 44 ocean columns present.
# ─────────────────────────────────────────────────────────────────────────────

N_PER_ARCH = 8_400
ARCHETYPES  = ['BOTNET_C2','BRUTE_FORCE','DOS_DDOS','EXPLOIT',
               'NORMAL','SCAN','THEFT_EXFIL']

def _tail_parquet(arch: str, n: int) -> pd.DataFrame:
    """Stream the last ``n`` rows from an archetype partition (sorted files)."""
    pdir  = OCEAN_DIR / f'ubt_archetype={arch}'
    files = sorted(pdir.glob('*.parquet'))
    if not files:
        raise FileNotFoundError(f'No parquet files under {pdir}')
    rows_needed = n
    chunks = []
    for f in reversed(files):          # walk backwards through time
        if rows_needed <= 0:
            break
        df_f = pd.read_parquet(f, engine='pyarrow')   # all 44 native columns
        take = df_f.tail(rows_needed)
        chunks.append(take)
        rows_needed -= len(take)
    # restore temporal order inside this archetype
    df_arch = pd.concat(reversed(chunks), ignore_index=True)
    df_arch['ubt_archetype'] = arch
    df_arch['ground_truth']  = 0 if arch == 'NORMAL' else 1
    return df_arch

print(f'Loading {N_PER_ARCH:,} rows × {len(ARCHETYPES)} archetypes …')
arch_dfs = []
for arch in ARCHETYPES:
    try:
        df_a = _tail_parquet(arch, N_PER_ARCH)
        arch_dfs.append(df_a)
        print(f'  {arch:15s}: {len(df_a):,} rows')
    except Exception as exc:
        print(f'  {arch:15s}: ERROR — {exc}')

test_df = pd.concat(arch_dfs, ignore_index=True)
ground_truth_all = test_df['ground_truth'].values.copy()
archetypes_all   = test_df['ubt_archetype'].values.copy()

# Feature frame: drop metadata cols that are NOT feature columns
feat_df = test_df.drop(columns=['ubt_archetype', 'ground_truth',
                                 'meta_src_ip', 'meta_dst_ip',
                                 'meta_timestamp', 'dataset_source',
                                 'univ_specific_attack'], errors='ignore')

print(f'\nTotal test slice : {len(test_df):,} rows')
print(f'  attack (1)     : {ground_truth_all.sum():,}')
print(f'  normal (0)     : {(ground_truth_all==0).sum():,}')
print(f'  feature columns: {feat_df.shape[1]}')


Loading 8,400 rows × 7 archetypes …
  BOTNET_C2      : 8,400 rows
  BRUTE_FORCE    : 8,400 rows
  DOS_DDOS       : 8,400 rows
  EXPLOIT        : 8,400 rows
  NORMAL         : 8,400 rows
  SCAN           : 8,400 rows
  THEFT_EXFIL    : 97 rows

Total test slice : 50,497 rows
  attack (1)     : 42,097
  normal (0)     : 8,400
  feature columns: 39


In [7]:

# ── Cell 6: End-to-End Pipeline Execution ─────────────────────────────────────
#
# Stage A – Batch vectorise + L2-normalise   (numpy only)
# Stage B – Batched ChromaDB RAG queries     (1 000 rows / call → avoids SQLite limit)
# Stage C – Sequential windowed inference    (AdaptiveTimeWindow + Dempster + quantum_fuse)
# ─────────────────────────────────────────────────────────────────────────────

RAG_BATCH = 1_000     # chromadb query batch size (SQLite variable limit: ~999)
BASE_TS   = 1_700_000_000.0   # synthetic epoch (s); each packet = +0.1 s

# ── Stage A: vectorise ────────────────────────────────────────────────────────
print('Stage A: vectorising …', end=' ', flush=True)
t0_vec = time.perf_counter()
X      = vectorize_v51(feat_df)        # (N, 114) float32
X_norm = _l2_normalise(X)              # unit-sphere
t1_vec = time.perf_counter()
N_total = len(X)
print(f'{N_total:,} vectors in {(t1_vec-t0_vec)*1e3:.1f} ms')

# ── Stage B: batch RAG (chunked to stay under SQLite variable limit) ──────────
n_batches = math.ceil(N_total / RAG_BATCH)
print(f'Stage B: querying ChromaDB ({n_batches} batches of ≤{RAG_BATCH}) …',
      end=' ', flush=True)
t0_rag = time.perf_counter()
rag_results: list = []
for b_start in range(0, N_total, RAG_BATCH):
    b_end   = min(b_start + RAG_BATCH, N_total)
    chunk   = X_norm[b_start:b_end]
    rag_results.extend(get_rag_context_batch(chunk, n_results=3))
t1_rag = time.perf_counter()
rag_time_s = t1_rag - t0_rag
print(f'done in {rag_time_s*1e3:.1f} ms  ({len(rag_results):,} results)')

# ── Stage C: sequential windowed inference ─────────────────────────────────────
print('Stage C: windowed evidence + quantum fusion …', end=' ', flush=True)

atw = AdaptiveTimeWindow(min_size_s=5.0, max_size_s=30.0)

results  : list = []
latencies: list = []

t0_seq = time.perf_counter()
for i, rag in enumerate(rag_results):
    ts   = BASE_TS + i * 0.1
    flow = FlowRecord(
        timestamp      = ts,
        vector         = X[i],
        top_similarity = rag['top_similarity'],
        top_archetype  = rag['top_archetype'],
        top_attack     = rag['top_attack'],
        packet_label   = archetypes_all[i],
        rag_context    = f"{rag['top_archetype']}:{rag['top_attack']}"
    )

    t_pkt0   = time.perf_counter()
    buffer   = atw.ingest(flow)
    evidence = accumulate_evidence(buffer)
    qf       = quantum_fuse(evidence)
    t_pkt1   = time.perf_counter()

    atw.adapt(qf['verdict'])
    latencies.append((t_pkt1 - t_pkt0) * 1e6)

    results.append({
        **qf,
        'ground_truth'  : int(ground_truth_all[i]),
        'ubt_archetype' : archetypes_all[i],
        'top_similarity': rag['top_similarity'],
        'top_archetype' : rag['top_archetype'],
    })

t1_seq = time.perf_counter()
seq_time_s = t1_seq - t0_seq

print(f'done in {seq_time_s*1e3:.1f} ms')
print(f'\nTotal inference time (B+C): {(rag_time_s+seq_time_s)*1e3:.1f} ms for {N_total:,} packets')

results_df = pd.DataFrame(results)
results_df['latency_us'] = latencies
print(results_df[['verdict','fused_probability','conflict_level','ground_truth','ubt_archetype']].head(10))


Stage A: vectorising … 50,497 vectors in 648.3 ms
Stage B: querying ChromaDB (51 batches of ≤1000) … done in 5024.2 ms  (50,497 results)
Stage C: windowed evidence + quantum fusion … done in 6181.6 ms

Total inference time (B+C): 11205.8 ms for 50,497 packets
    verdict  fused_probability  conflict_level  ground_truth ubt_archetype
0  CRITICAL           0.950000           0.000             1     BOTNET_C2
1  CRITICAL           0.980779           0.038             1     BOTNET_C2
2  CRITICAL           0.981995           0.038             1     BOTNET_C2
3  CRITICAL           0.981821           0.038             1     BOTNET_C2
4  CRITICAL           0.981757           0.038             1     BOTNET_C2
5  CRITICAL           0.981744           0.038             1     BOTNET_C2
6  CRITICAL           0.981743           0.038             1     BOTNET_C2
7  CRITICAL           0.981744           0.038             1     BOTNET_C2
8  CRITICAL           0.981745           0.038             1     

In [8]:

# ── Diagnostic: RAG similarity distribution for NORMAL traffic ────────────────
#
# Understand why the 75% floor didn't fix NORMAL F1:
#   How many NORMAL packets match NORMAL vs. attack medoids?
#   What is the similarity distribution for NORMAL packets?
# ─────────────────────────────────────────────────────────────────────────────

normal_mask = results_df['ubt_archetype'] == 'NORMAL'
normal_results = results_df[normal_mask]

print('=' * 60)
print('  DIAGNOSTIC: RAG results for NORMAL (ground-truth=0) packets')
print('=' * 60)
print(f'  Total NORMAL packets: {normal_mask.sum():,}')
print()

# Top archetype distribution
arch_dist = normal_results['top_archetype'].value_counts()
print('  RAG top_archetype distribution for NORMAL packets:')
for arch, cnt in arch_dist.items():
    print(f'    {arch:20s}: {cnt:6,}  ({100*cnt/normal_mask.sum():.1f}%)')

print()

# Similarity distribution
normal_sims = normal_results['top_similarity']
print(f'  Similarity stats for NORMAL packets:')
print(f'    mean : {normal_sims.mean():.2f}%')
print(f'    p25  : {normal_sims.quantile(0.25):.2f}%')
print(f'    p50  : {normal_sims.quantile(0.50):.2f}%')
print(f'    p75  : {normal_sims.quantile(0.75):.2f}%')
print(f'    min  : {normal_sims.min():.2f}%')
print(f'    max  : {normal_sims.max():.2f}%')
print(f'    < 75%: {(normal_sims < 75).sum():,}  ({100*(normal_sims < 75).sum()/len(normal_sims):.1f}%)')
print(f'    >= 75%: {(normal_sims >= 75).sum():,}  ({100*(normal_sims >= 75).sum()/len(normal_sims):.1f}%)')

print()

# Verdict distribution for NORMAL packets
verd_dist = normal_results['verdict'].value_counts()
print('  Verdict distribution for NORMAL packets:')
for verd, cnt in verd_dist.items():
    print(f'    {verd:12s}: {cnt:6,}  ({100*cnt/normal_mask.sum():.1f}%)')

print()

# How many NORMAL packets have UNKNOWN archetype after the floor?
unk_normal = (normal_results['top_archetype'] == 'UNKNOWN').sum()
print(f'  NORMAL packets floored to UNKNOWN: {unk_normal:,} ({100*unk_normal/normal_mask.sum():.1f}%)')

# How many NORMAL packets are matched to NORMAL archetype medoids?
normal_matched_normal = (normal_results['top_archetype'].isin(NORMAL_ARCHETYPES)).sum()
print(f'  NORMAL packets matched to NORMAL medoids: {normal_matched_normal:,} ({100*normal_matched_normal/normal_mask.sum():.1f}%)')


  DIAGNOSTIC: RAG results for NORMAL (ground-truth=0) packets
  Total NORMAL packets: 8,400

  RAG top_archetype distribution for NORMAL packets:
    NORMAL              :  8,133  (96.8%)
    THEFT_EXFIL         :     70  (0.8%)
    DOS_DDOS            :     68  (0.8%)
    SCAN                :     59  (0.7%)
    UNKNOWN             :     41  (0.5%)
    BOTNET_C2           :     21  (0.2%)
    EXPLOIT             :      5  (0.1%)
    BRUTE_FORCE         :      3  (0.0%)

  Similarity stats for NORMAL packets:
    mean : 98.96%
    p25  : 100.00%
    p50  : 100.00%
    p75  : 100.00%
    min  : 0.00%
    max  : 100.00%
    < 75%: 41  (0.5%)
    >= 75%: 8,359  (99.5%)

  Verdict distribution for NORMAL packets:
    CLEAR       :  8,133  (96.8%)
    CRITICAL    :    180  (2.1%)
    ELEVATED    :     62  (0.7%)

  NORMAL packets floored to UNKNOWN: 41 (0.5%)
  NORMAL packets matched to NORMAL medoids: 8,133 (96.8%)


In [9]:

# ── Cell 7: Binary Classification Metrics ─────────────────────────────────────
#
# Mapping:
#   CRITICAL / WARNING / ELEVATED  →  pred = 1  (attack)
#   MONITOR  / CLEAR               →  pred = 0  (benign)
# ─────────────────────────────────────────────────────────────────────────────

ATTACK_VERDICTS = {'CRITICAL', 'WARNING', 'ELEVATED'}

y_pred  = results_df['verdict'].apply(lambda v: 1 if v in ATTACK_VERDICTS else 0).values
y_truth = results_df['ground_truth'].values

acc = accuracy_score(y_truth, y_pred)
pre = precision_score(y_truth, y_pred, zero_division=0)
rec = recall_score(y_truth, y_pred,    zero_division=0)
f1  = f1_score(y_truth, y_pred,        zero_division=0)
cm  = confusion_matrix(y_truth, y_pred)

print('=' * 56)
print('  PHASE 4.1 — BINARY CLASSIFICATION METRICS')
print('=' * 56)
print(f'  Packets evaluated : {N_total:,}')
print(f'  Accuracy          : {acc:.4f}')
print(f'  Precision         : {pre:.4f}')
print(f'  Recall            : {rec:.4f}')
print(f'  F1-Score          : {f1:.4f}')
print()
print('Confusion Matrix  (rows=truth, cols=pred)')
print('               Pred-0   Pred-1')
print(f'  Truth-0 (N) : {cm[0,0]:7,}  {cm[0,1]:7,}')
print(f'  Truth-1 (A) : {cm[1,0]:7,}  {cm[1,1]:7,}')
print()
print(classification_report(y_truth, y_pred,
                             target_names=['benign','attack'], zero_division=0))

# ── Per-archetype breakdown ────────────────────────────────────────────────────
# For NORMAL (ground truth = 0): use pos_label=0 since all truth values are 0.
# For ATTACK archetypes (ground truth = 1): use default pos_label=1.
print('Per-archetype metrics:')
print(f'  {"Archetype":15s}  {"F1":>6s}  {"Acc":>6s}  {"Correct":>7s}  {"Total":>5s}')
print(f'  {"-"*15}  {"-"*6}  {"-"*6}  {"-"*7}  {"-"*5}')
for arch in ARCHETYPES:
    mask  = results_df['ubt_archetype'] == arch
    yt, yp = y_truth[mask], y_pred[mask]
    gt_label = 0 if arch == 'NORMAL' else 1
    f1_a  = f1_score(yt, yp, zero_division=0, pos_label=gt_label)
    acc_a = accuracy_score(yt, yp)
    correct = (yt == yp).sum()
    print(f'  {arch:15s}  {f1_a:.4f}  {acc_a:.4f}  {correct:7,}  {mask.sum():5,}')


  PHASE 4.1 — BINARY CLASSIFICATION METRICS
  Packets evaluated : 50,497
  Accuracy          : 0.9681
  Precision         : 0.9935
  Recall            : 0.9681
  F1-Score          : 0.9806

Confusion Matrix  (rows=truth, cols=pred)
               Pred-0   Pred-1
  Truth-0 (N) :   8,133      267
  Truth-1 (A) :   1,343   40,754

              precision    recall  f1-score   support

      benign       0.86      0.97      0.91      8400
      attack       0.99      0.97      0.98     42097

    accuracy                           0.97     50497
   macro avg       0.93      0.97      0.95     50497
weighted avg       0.97      0.97      0.97     50497

Per-archetype metrics:
  Archetype            F1     Acc  Correct  Total
  ---------------  ------  ------  -------  -----
  BOTNET_C2        0.9642  0.9310    7,820  8,400
  BRUTE_FORCE      1.0000  1.0000    8,400  8,400
  DOS_DDOS         1.0000  1.0000    8,400  8,400
  EXPLOIT          0.9527  0.9096    7,641  8,400
  NORMAL           0

In [10]:

# ── Cell 8: Performance Profiling Report ──────────────────────────────────────
_lats = np.array(latencies)           # per-packet sequential stage (μs)
rag_per_pkt_us = (rag_time_s / N_total) * 1e6
total_per_pkt_us = ((rag_time_s + seq_time_s) / N_total) * 1e6
pps = N_total / (rag_time_s + seq_time_s)

print('=' * 56)
print('  PHASE 4.1 — PERFORMANCE PROFILING')
print('=' * 56)
print(f'  Total packets           : {N_total:,}')
print(f'  RAG batch time          : {rag_time_s*1e3:.2f} ms')
print(f'  Sequential fusion time  : {seq_time_s*1e3:.2f} ms')
print(f'  Combined time           : {(rag_time_s+seq_time_s)*1e3:.2f} ms')
print()
print(f'  RAG cost / packet       : {rag_per_pkt_us:.2f} μs')
print()
print(f'  Sequential stage stats  (μs/pkt):')
print(f'    mean  : {_lats.mean():.2f}')
print(f'    p50   : {np.percentile(_lats, 50):.2f}')
print(f'    p95   : {np.percentile(_lats, 95):.2f}')
print(f'    p99   : {np.percentile(_lats, 99):.2f}')
print(f'    max   : {_lats.max():.2f}')
print()
print(f'  End-to-end mean latency : {total_per_pkt_us:.2f} μs/pkt')
print(f'  Throughput              : {pps:,.0f} packets/sec')

# Target checks
LATENCY_TARGET_US  = 1_000.0
PPS_TARGET         = 10_000

lat_ok = total_per_pkt_us < LATENCY_TARGET_US
pps_ok = pps >= PPS_TARGET
print()
print(f'  Latency < {LATENCY_TARGET_US:,.0f} μs   : {"✅ PASS" if lat_ok else "❌ FAIL"}  ({total_per_pkt_us:.2f} μs)')
print(f'  PPS ≥ {PPS_TARGET:,}          : {"✅ PASS" if pps_ok else "❌ FAIL"}  ({pps:,.0f} pps)')


  PHASE 4.1 — PERFORMANCE PROFILING
  Total packets           : 50,497
  RAG batch time          : 5024.17 ms
  Sequential fusion time  : 6181.58 ms
  Combined time           : 11205.75 ms

  RAG cost / packet       : 99.49 μs

  Sequential stage stats  (μs/pkt):
    mean  : 119.25
    p50   : 139.50
    p95   : 219.90
    p99   : 416.50
    max   : 8951.90

  End-to-end mean latency : 221.91 μs/pkt
  Throughput              : 4,506 packets/sec

  Latency < 1,000 μs   : ✅ PASS  (221.91 μs)
  PPS ≥ 10,000          : ❌ FAIL  (4,506 pps)


In [11]:

# ── Cell 9: Quantum Advantage Validation (K > 0.50) ───────────────────────────
#
# High-conflict cases (Dempster K > 0.50) are where quantum interference
# should help most.  Compare quantum-fused F1 vs a naïve RAG-similarity
# baseline that thresholds on top_similarity alone.
# ─────────────────────────────────────────────────────────────────────────────

K_THRESHOLD      = 0.50
RAG_SIM_THRESHOLD = 70.0   # similarity % above which we say "attack"

conflict_arr = results_df['conflict_level'].values
sim_arr      = results_df['top_similarity'].values
hc_mask      = conflict_arr > K_THRESHOLD

n_hc = hc_mask.sum()
print(f'High-conflict packets (K > {K_THRESHOLD}): {n_hc:,} / {N_total:,}  '
      f'({100*n_hc/N_total:.1f}%)')

if n_hc == 0:
    print('No high-conflict packets found — quantum advantage test skipped.')
else:
    yt_hc = y_truth[hc_mask]
    yp_hc = y_pred[hc_mask]

    # RAG-only baseline: attack if similarity > threshold, else benign
    rag_pred_hc = (sim_arr[hc_mask] > RAG_SIM_THRESHOLD).astype(int)

    f1_quantum = f1_score(yt_hc, yp_hc,      zero_division=0)
    f1_rag     = f1_score(yt_hc, rag_pred_hc, zero_division=0)
    delta_f1   = f1_quantum - f1_rag

    print()
    print('=' * 56)
    print('  QUANTUM ADVANTAGE (K > 0.50 subset)')
    print('=' * 56)
    print(f'  Subset size         : {n_hc:,}')
    print(f'  Quantum-fused  F1   : {f1_quantum:.4f}')
    print(f'  RAG-only       F1   : {f1_rag:.4f}   '
          f'(threshold: sim > {RAG_SIM_THRESHOLD}%)')
    print(f'  Δ F1 (quantum−RAG)  : {delta_f1:+.4f}')
    advantage = delta_f1 > 0
    print(f'  Quantum advantage   : {"✅ YES" if advantage else "❌ NO"}')

    # Interference type breakdown in high-conflict subset
    itype_counts = results_df[hc_mask]['interference_type'].value_counts()
    print()
    print('  Interference types in high-conflict subset:')
    for itype, cnt in itype_counts.items():
        print(f'    {itype:15s}: {cnt:,}  ({100*cnt/n_hc:.1f}%)')


High-conflict packets (K > 0.5): 14,497 / 50,497  (28.7%)

  QUANTUM ADVANTAGE (K > 0.50 subset)
  Subset size         : 14,497
  Quantum-fused  F1   : 0.9598
  RAG-only       F1   : 0.9821   (threshold: sim > 70.0%)
  Δ F1 (quantum−RAG)  : -0.0223
  Quantum advantage   : ❌ NO

  Interference types in high-conflict subset:
    DESTRUCTIVE    : 9,819  (67.7%)
    MIXED          : 3,089  (21.3%)
    CONSTRUCTIVE   : 1,589  (11.0%)


In [12]:

# ── Cell 10: Phase 4.1 Completion Summary ─────────────────────────────────────
print('╔══════════════════════════════════════════════════════╗')
print('║     PHASE 4.1 — SYSTEM EVALUATION COMPLETE          ║')
print('╠══════════════════════════════════════════════════════╣')
print(f'║  Test slice          : {N_total:,} packets ({len(ARCHETYPES)} archetypes)'.ljust(55) + '║')
print(f'║  Accuracy            : {acc:.4f}'.ljust(55) + '║')
print(f'║  Precision           : {pre:.4f}'.ljust(55) + '║')
print(f'║  Recall              : {rec:.4f}'.ljust(55) + '║')
print(f'║  F1-Score            : {f1:.4f}'.ljust(55) + '║')
print(f'║  Mean latency        : {total_per_pkt_us:.2f} μs/pkt'.ljust(55) + '║')
print(f'║  Throughput          : {pps:,.0f} pps'.ljust(55) + '║')
if n_hc > 0:
    print(f'║  Quantum ΔF1 (K>0.5) : {delta_f1:+.4f}'.ljust(55) + '║')
print('╠══════════════════════════════════════════════════════╣')
print('║  Pipeline stages proven end-to-end:                 ║')
print('║    ✅ vectorize_v51  (114-dim float32)               ║')
print('║    ✅ get_rag_context_batch  (ChromaDB cosine)        ║')
print('║    ✅ AdaptiveTimeWindow  (Phase 3.1)                ║')
print('║    ✅ accumulate_evidence (Dempster-Shafer, 3.2)     ║')
print('║    ✅ quantum_fuse  (H-gate interference, 3.3)       ║')
print('╚══════════════════════════════════════════════════════╝')


╔══════════════════════════════════════════════════════╗
║     PHASE 4.1 — SYSTEM EVALUATION COMPLETE          ║
╠══════════════════════════════════════════════════════╣
║  Test slice          : 50,497 packets (7 archetypes) ║
║  Accuracy            : 0.9681                        ║
║  Precision           : 0.9935                        ║
║  Recall              : 0.9681                        ║
║  F1-Score            : 0.9806                        ║
║  Mean latency        : 221.91 μs/pkt                 ║
║  Throughput          : 4,506 pps                     ║
║  Quantum ΔF1 (K>0.5) : -0.0223                       ║
╠══════════════════════════════════════════════════════╣
║  Pipeline stages proven end-to-end:                 ║
║    ✅ vectorize_v51  (114-dim float32)               ║
║    ✅ get_rag_context_batch  (ChromaDB cosine)        ║
║    ✅ AdaptiveTimeWindow  (Phase 3.1)                ║
║    ✅ accumulate_evidence (Dempster-Shafer, 3.2)     ║
║    ✅ quantum_fuse  (H-gate interfe